In [1]:
import boto3
import json
import os
#import image processing from backend
from image_processing import * 

import dlib
import cv2
import pandas as pd
from pathlib import Path
import pandas as pd
import numpy as np
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

def get_metadata(bucket_name, metadata_file_key, s3_client):
    try:
        metadata_object = s3_client.get_object(Bucket=bucket_name, Key=metadata_file_key)
        metadata_content = metadata_object['Body'].read().decode('utf-8')
        metadata = json.loads(metadata_content)
        return metadata
    except Exception as e:
        print(f"Error retrieving metadata from S3: {e}")
        return None


In [2]:
def get_camera_info(camera_info):
    camera_matrix = np.array(camera_info[0], dtype='double')
    dist_coeffs = np.array(camera_info[1], dtype='double')
    return camera_matrix, dist_coeffs


In [3]:
def pre_process_image(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    dlib_faces = detector(gray)

    processed_data = []

    for dlib_face in dlib_faces:
        shape = predictor(gray, dlib_face)

        for (i, (start, end)) in enumerate([(36,42), (42,48)]):
            eye_image, (eye_min_x, eye_min_y, eye_max_x, eye_max_y) = extract_eye_region(gray, shape, range(start, end))
            eye_image_b = convert_eye_to_binary(eye_image)
            pupil_center, _ = detect_pupil(eye_image_b)

            if pupil_center:
                pupil_center_global = (pupil_center[0] + eye_min_x, pupil_center[1] + eye_min_y)
                pupil_center_global = tuple(pc.item() if isinstance(pc, np.generic) else pc for pc in pupil_center_global)
                bounding_box = (eye_min_x, eye_min_y, eye_max_x - eye_min_x, eye_max_y - eye_min_y)
                bounding_box = tuple(bb.item() if isinstance(bb, np.generic) else bb for bb in bounding_box)


                eye_data = {
                    'eye_position': 'left' if i == 0 else 'right',
                    'pupil_center': pupil_center_global,
                    'bounding_box': bounding_box
                }
                processed_data.append(eye_data)

                left_eye_info = None
                right_eye_info = None
                left_eye_bbox = None
                right_eye_bbox = None

                for eye_data in processed_data:
                    if eye_data['eye_position'] == 'left':
                        left_eye_info = eye_data['pupil_center']
                        left_eye_bbox = eye_data['bounding_box']
                    else:
                        right_eye_info = eye_data['pupil_center']
                        right_eye_bbox = eye_data['bounding_box']
        break
    return processed_data, left_eye_info, right_eye_info, left_eye_bbox, right_eye_bbox, shape

In [4]:
def format_calibration_data_row(calibratoin_points, left_eye_info, right_eye_info, left_eye_bbox, right_eye_bbox, head_pose):
    rotation_vector, translation_vector = head_pose if head_pose else (np.zeros((3, 1)), np.zeros((3, 1)))

    rotation_vector_str = ','.join(map(str, rotation_vector.flatten()))
    translation_vector_str = ','.join(map(str, translation_vector.flatten()))

    data_row = [
        calibratoin_points[0], calibratoin_points[1],
        left_eye_info[0], left_eye_info[1],
        *left_eye_bbox,
        right_eye_info[0], right_eye_info[1],
        *right_eye_bbox,
        rotation_vector_str, translation_vector_str
    ]
    return data_row

def format_eye_gaze_data_row(cursors_position, left_eye_info, right_eye_info, left_eye_bbox, right_eye_bbox, head_pose):
    rotation_vector, translation_vector = head_pose if head_pose else (np.zeros((3, 1)), np.zeros((3, 1)))

    rotation_vector_str = ','.join(map(str, rotation_vector.flatten()))
    translation_vector_str = ','.join(map(str, translation_vector.flatten()))

    data_row = [
        cursors_position[0], cursors_position[1],
        left_eye_info[0], left_eye_info[1],
        *left_eye_bbox,
        right_eye_info[0], right_eye_info[1],
        *right_eye_bbox,
        rotation_vector_str, translation_vector_str
    ]
    return data_row

In [5]:
def create_and_replace_csv(local_base_dir, subdirectory, csv_file_name, image_data):
    # Path for the CSV file within the subdirectory
    csv_dir_path = os.path.join(local_base_dir, subdirectory)
    os.makedirs(csv_dir_path, exist_ok=True)  # Ensure the subdirectory exists
    csv_path = os.path.join(csv_dir_path, csv_file_name)
    
    # Create a new DataFrame for the CSV data
    new_data_df = pd.DataFrame(image_data)
    
    # Write the new DataFrame to the CSV file, overwriting the old data
    new_data_df.to_csv(csv_path, index=False, header=False)


In [6]:
def get_image_paths(bucket_name, key_prefix, subdirectory, s3_client):
    paginator = s3_client.get_paginator('list_objects_v2')
    image_paths = []

    # List objects within a specific subdirectory
    for page in paginator.paginate(Bucket=bucket_name, Prefix=f"{key_prefix}{subdirectory}"):
        for obj in page.get('Contents', []):
            # Skip directories
            if obj['Key'].endswith('/'):
                continue
            image_paths.append(obj['Key'])

    return image_paths

In [7]:
import pandas as pd

def process_images(image_paths, local_base_dir, subdirectory, csv_file_name, camera_info):
    # This will hold all new rows for the CSV file
    image_data = []
    
    # Path to the current CSV file
    current_csv_path = os.path.join(local_base_dir, subdirectory, csv_file_name)
    
    # Check if the CSV file already exists and read the necessary columns
    if os.path.exists(current_csv_path):
        current_data_df = pd.read_csv(current_csv_path, header=None)

        existing_data = current_data_df.iloc[:, 1:3].values
    else:
        existing_data = []

    for index, image_path in enumerate(image_paths):
        # Assuming the image_path is just the file name, construct the full path
        full_image_path = os.path.join(local_base_dir, image_path)
        image = cv2.imread(full_image_path)
        
        # Process the image
        processed_data, left_eye_info, right_eye_info, left_eye_bbox, right_eye_bbox, shape = pre_process_image(image)
        
        # Get head pose data
        camera_matrix, dist_coeffs = camera_info
        head_pose = get_head_pose(shape, camera_matrix, dist_coeffs)
        
        if index < len(existing_data):
            cursor_or_calibration = existing_data[index]
        else:
            cursor_or_calibration = [np.nan, np.nan]  # or some default value
        
        # Format the data row for CSV, including the existing data
        if 'calibration' in image_path:
            data_row = format_calibration_data_row(cursor_or_calibration, left_eye_info, right_eye_info, left_eye_bbox, right_eye_bbox, head_pose)
        else:
            data_row = format_eye_gaze_data_row(cursor_or_calibration, left_eye_info, right_eye_info, left_eye_bbox, right_eye_bbox, head_pose)
        
        # Append to the list of image data
        image_data.append([image_path] + data_row)

    # Create and replace the CSV file with the new data
    create_and_replace_csv(local_base_dir, subdirectory, csv_file_name, image_data)

In [10]:

import json
import os
from botocore.exceptions import ClientError

def get_metadata(bucket_name, metadata_file_key, s3_client):
    try:
        metadata_object = s3_client.get_object(Bucket=bucket_name, Key=metadata_file_key)
        metadata_content = metadata_object['Body'].read().decode('utf-8')
        metadata = json.loads(metadata_content)
        return metadata
    except Exception as e:
        print(f"Error retrieving metadata from S3: {e}")
        return None

def download_data(key_prefix, local_base_dir, s3_client, bucket_name):
    paginator = s3_client.get_paginator('list_objects_v2')
    for page in paginator.paginate(Bucket=bucket_name, Prefix=key_prefix):
        for obj in page.get('Contents', []):
            s3_object_key = obj['Key']
            if s3_object_key.endswith('/'):
                continue
            local_file_path = os.path.join(local_base_dir, s3_object_key)
            os.makedirs(os.path.dirname(local_file_path), exist_ok=True)

            try:
                local_file_info = os.stat(local_file_path)
                s3_object_info = s3_client.head_object(Bucket=bucket_name, Key=s3_object_key)
                if local_file_info.st_size == s3_object_info['ContentLength']:
                    print(f"File {s3_object_key} already exists locally and is up to date. Skipping download.")
                    continue
            except (FileNotFoundError, ClientError):
                pass

            s3_client.download_file(bucket_name, s3_object_key, local_file_path)
            print(f"Downloaded {s3_object_key} to {local_file_path}")

def process_data_if_needed(key_prefix, local_base_dir, s3_client, bucket_name):
    # Retrieve metadata and determine if processing is needed
    metadata_key = f"{key_prefix}metadata.json"
    metadata_object = s3_client.get_object(Bucket=bucket_name, Key=metadata_key)
    metadata_content = metadata_object['Body'].read().decode('utf-8')
    metadata = json.loads(metadata_content)

    # If cameraInfo is present in metadata, process the data
    if 'cameraInfo' in metadata:
        print(f"Processing data in {key_prefix}")
        camera_matrix, dist_coeffs = get_camera_info(metadata['cameraInfo'])
        
        # Get the list of image paths that need processing
        calibration_image_paths = get_image_paths(bucket_name, key_prefix, 'calibration_images/', s3_client)
        eye_gaze_image_paths = get_image_paths(bucket_name, key_prefix, 'eye_gaze_images/', s3_client)
        
        subdir_prefix = key_prefix.rstrip('/')  # Ensure the prefix doesn't end with a '/'
        
        # Process calibration images
        if calibration_image_paths:
            process_images(calibration_image_paths, local_base_dir, subdir_prefix, 'calibration_data.csv', camera_info=(camera_matrix, dist_coeffs))
        
        # Process eye gaze images
        if eye_gaze_image_paths:
            process_images(eye_gaze_image_paths, local_base_dir, subdir_prefix, 'eye_gaze_data.csv', camera_info=(camera_matrix, dist_coeffs))
        
    else:
        print(f"No processing needed for {key_prefix}")


def process_s3_bucket_data(bucket_name, local_base_dir):
    s3_client = boto3.client('s3')
    
    paginator = s3_client.get_paginator('list_objects_v2')
    print(f"Retrieving all S3 objects from bucket: {bucket_name}")
    page_iterator = paginator.paginate(Bucket=bucket_name, Prefix='data/')

    for page in page_iterator:
        for content in page.get('Contents', []):
            key = content['Key']
            if key.endswith('/metadata.json'):
                subdir_prefix = '/'.join(key.split('/')[:-1]) + '/'
                print(f"Processing data in {subdir_prefix}")
                download_data(subdir_prefix, local_base_dir, s3_client, bucket_name)
                process_data_if_needed(subdir_prefix, local_base_dir, s3_client, bucket_name)
            else:
                print(f"Encountered non-metadata key: {key}")



In [11]:
bucket_name = 'eye-gaze-data'
local_base_dir = './'

os.makedirs(local_base_dir, exist_ok=True)

process_s3_bucket_data(bucket_name, local_base_dir)

Retrieving all S3 objects from bucket: eye-gaze-data
Encountered non-metadata key: data/Hossein/calibration_data.csv
Encountered non-metadata key: data/Hossein/calibration_images/Hossein_04abe5a2-0e23-4e8c-87a5-5dbb8e48ba55.png
Encountered non-metadata key: data/Hossein/calibration_images/Hossein_08cb8659-ee4e-44d9-9547-6260ac36c5ce.png
Encountered non-metadata key: data/Hossein/calibration_images/Hossein_0b8c9fa8-7184-44ed-bc15-af08563f6e17.png
Encountered non-metadata key: data/Hossein/calibration_images/Hossein_0eef3086-8737-4426-b427-2069c3c38e22.png
Encountered non-metadata key: data/Hossein/calibration_images/Hossein_19561d89-1c54-4f78-91d2-41ae8405974c.png
Encountered non-metadata key: data/Hossein/calibration_images/Hossein_24eaef3e-24f3-46df-8738-19cb73a20327.png
Encountered non-metadata key: data/Hossein/calibration_images/Hossein_2990d0e8-73be-49b6-a63d-5800f4842e6d.png
Encountered non-metadata key: data/Hossein/calibration_images/Hossein_31e81440-74de-4433-a639-2afd728022ca